In [1]:
import string
import re
import nltk
import spacy

# nltk.download()
nlp = spacy.load('en_core_web_sm')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


## Lowercasing

In [2]:
text_with_upper = "Hello, POLAND is a Very BeautiFul CountrY!"
text_with_upper.lower()

'hello, poland is a very beautiful country!'

## Punctuation Removal

### **Most of them use `string.punctuation`**



Based on the [StackOverflow question](https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string)

I ordered the solutions from the fastest to the slowest

In [3]:
puncts_text = "HI!!! I overuse ... !(@*punctuations +-*/ and*(& other signs __*(&!!"

### using `str.translate()` 
Probably the fastest way

In [4]:
puncts_text.translate(str.maketrans('', '', string.punctuation))

'HI I overuse  punctuations  and other signs '

### Using `re`

`re` stands for Regular Expressions

#### Method 1 with string.punctuation

In [5]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
regex.sub('', puncts_text)

'HI I overuse  punctuations  and other signs '

#### Method 1 in a single line

In [6]:
re.sub(f'[{re.escape(string.punctuation)}]','', puncts_text)

'HI I overuse  punctuations  and other signs '

#### Method 2

Removes **not words** and **not spaces**

Note: `re` treats underscore as a word so its results are different

In [7]:
re.sub(r'[^\w\s]','', puncts_text)

'HI I overuse  punctuations  and other signs __'

### Excluding string.punctuation

In [8]:
# a slower solution
exclude = set(string.punctuation)
"".join(ch for ch in puncts_text if ch not in exclude)

'HI I overuse  punctuations  and other signs '

### Using `str.replace()`

In [9]:
clean_text = puncts_text
for c in string.punctuation:
    clean_text = clean_text.replace(c, "")
clean_text

'HI I overuse  punctuations  and other signs '

## Numbers Removal



In [10]:
text_numbers = '12abcd405'

### Using `str.translate` with `string.digits`

In [11]:
text_numbers.translate(str.maketrans('', '', string.digits))

'abcd'

### Using `re`

In [12]:
re.sub(r'\d+', '', text_numbers)

'abcd'

In [13]:
re.sub(r'[0-9]+', '', text_numbers)

'abcd'

### Using `join()` and NOT `isdigit()`

In [14]:
"".join(i for i in text_numbers if not i.isdigit())

'abcd'

### Using `join()` and `isalpha()`

This actually isn't the correct solution, because `isalpha()` is True only for letters.

In [15]:
"".join(i for i in text_numbers if i.isalpha())

'abcd'

## HTML Tags Removal

Solutions from [Stack Overflow](https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python)

In [16]:
html_text = """<tr class="color-5 negri a-bottom">
<td class="a-center" width="11%"><div style="min-width: 80px">3-Pointers</div></td>
<td><div class="left" style="min-width: 120px; max-width:175px; width: 57%">
<div class="left margen-l2">Player</div>
<div class="right"> Team</div>
</div>
</td>
<td><div style="min-width: 60px; ">Season</div></td>
<td><div class="">W/L Game</div>
</td>
</tr>"""


### Using `re`

 - begin with tag opening '<'
 - then not '<'
 - not '<' at least once

In [17]:
re.sub('<[^<]+?>', '', html_text)

'\n3-Pointers\n\nPlayer\n Team\n\n\nSeason\nW/L Game\n\n'

### Using `BeautifulSoup`

 - `get_text()` removes HTML tags
 - `strip = True` removes whitespaces and newlines

In [18]:
from bs4 import BeautifulSoup

In [19]:
soup = BeautifulSoup(html_text, 'html.parser')
soup.get_text(",", strip=True)

'3-Pointers,Player,Team,Season,W/L Game'

### Extreme example

In [20]:
html_comment = "<img<!-- --> src=x onerror=alert(1);//><!-- -->"

Both solutions fail

In [21]:
re.sub('<[^<]+?>', '', html_comment)

'<img src=x onerror=alert(1);//>'

In [22]:
soup = BeautifulSoup(html_comment, 'html.parser')
soup.get_text(",", strip=True)

'src=x onerror=alert(1);//>'

In [23]:
tag_re = re.compile(r'(<!--.*?-->|<[^>]*>)')
no_tags = tag_re.sub('', html_comment)
no_tags

' src=x onerror=alert(1);//>'

In [24]:
import html

html.escape(no_tags)

' src=x onerror=alert(1);//&gt;'

## URL Removal

From this [StackOverflow Question](https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python/11332580)

In [25]:
text_url = """text1
text2
http://url.com/bla1/blah1/
text3
text4
http://url.com/bla2/blah2/
text5
text6
http://url.com/bla3/blah3/"""

In [26]:
re.sub(r'http\S+', '', text_url)

'text1\ntext2\n\ntext3\ntext4\n\ntext5\ntext6\n'

## Newlines, spaces and tabs removal

### Using `str.split()` and `join()`

In [27]:
my_str="I want to Remove all white \t\n\n\r spaces, new lines \n and tabs \t"
" ".join(my_str.split())

'I want to Remove all white spaces, new lines and tabs'

### Using `re`

 - **`\s` stands for whitespace character, equivalent to `[ \n\r\t\f]`**
 - **`\S` stands for not whitespace character, equivalent to `[^\s]`**

In [28]:
re.sub('\s+', ' ', my_str)

'I want to Remove all white spaces, new lines and tabs '

In [29]:
re.sub('[^\S]+', ' ', my_str)

'I want to Remove all white spaces, new lines and tabs '

In [30]:
re.sub('[\t\n\r\f ]+', ' ', my_str)

'I want to Remove all white spaces, new lines and tabs '

### Using `re.findall()`

Taken from [StackOverflow](https://stackoverflow.com/questions/4697882/how-can-i-find-all-matches-to-a-regular-expression-in-python)

NOTE: I believe this approach is slower than the one with `re.sub()`. 

In [31]:
match = re.findall('[\w]+ ', my_str)
"".join(match)

'I want to Remove all white new lines and tabs '

## Emojis Removal

Found [here](https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b#gistcomment-3315605)

In [32]:
text_emojis = u"Hi 🤔 How is your 🙈 and 😌. Have a nice weekend 💕👭👙\U0001F600\U0001F300"
text_emojis

'Hi 🤔 How is your 🙈 and 😌. Have a nice weekend 💕👭👙😀🌀'

In [33]:
emojis_pattern = re.compile(pattern="["
                    u"\U0001F600-\U0001F64F"  # emoticons
                    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                    u"\U0001F680-\U0001F6FF"  # transport & map symbols
                    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                    u"\U00002500-\U00002BEF"  # chinese char
                    u"\U00002702-\U000027B0"
                    u"\U00002702-\U000027B0"
                    u"\U000024C2-\U0001F251"
                    u"\U0001f926-\U0001f937"
                    u"\U00010000-\U0010ffff"
                    u"\u2640-\u2642"
                    u"\u2600-\u2B55"
                    u"\u200d"
                    u"\u23cf"
                    u"\u23e9"
                    u"\u231a"
                    u"\ufe0f"  # dingbats
                    u"\u3030"
                "]+", flags = re.UNICODE)

emojis_pattern.sub(r'', text_emojis)

'Hi  How is your  and . Have a nice weekend '

## Replacing Accented Characters

In [34]:
import unidecode
text_accented = "Málaga, àéêöhello. Polish: ńŃćĆśŚęąóżŻźŹ letters. German üöäöß letters"

unidecode.unidecode(text_accented)

'Malaga, aeeohello. Polish: nNcCsSeaozZzZ letters. German uoaoss letters'

## Spelling corrections

Solutions from [StackOverflow](https://stackoverflow.com/questions/13928155/spell-checker-for-python)

### Using `spellchecker`

In [35]:
from spellchecker import SpellChecker

spell = SpellChecker()

def correct_spelling(text):
    corrected_text = list()
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        next_word = word
        if word in misspelled_words:
            next_word = spell.correction(word)
        corrected_text.append(next_word)
    
    return " ".join(corrected_text)

text_misspelled = "I realli needt smoe corection. This sentnce has mispelled wirds"
correct_spelling(text_misspelled)

'I really need some corrections This sentence has misspelled words'

### Using `autocorrect`

In [36]:
from autocorrect import Speller

speller = Speller(lang='en')

print(speller(text_misspelled))

I really need some correction. This sentence has misspelled words


### Using `textblob`

Found [here](https://www.geeksforgeeks.org/python-textblob-correct-method/)

In [37]:
from textblob import TextBlob

TextBlob(text_misspelled).correct()

TextBlob("I really need some correction. His sentence has dispelled words")

## Reading a text file

### Reading line by line

In [38]:
with open('metamorphosis.txt', 'r') as f:
    # without newlines
    lines = [line.rstrip() for line in f]
    # read text line by line including newlines \n
    # text_lines = f.readlines()

N_LINES = 20
text_lines = " ".join([line for line in lines[:N_LINES]])
text_lines

'One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin.  He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections.  The bedding was hardly able to cover it and seemed ready to slide off any moment.  His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked.  "What\'s happened to me?" he thought.  It wasn\'t a dream.  His room, a proper human room although a little too small, lay peacefully between its four familiar walls.  A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame.  It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff that covered the whole of her lower ar

### Reading the whole text

In [39]:
# read the whole file
with open('metamorphosis.txt', 'r') as f:
    text = f.read()

N_CHARS = 300
text_chars = text[:N_CHARS]
text_chars

'One morning, when Gregor Samsa woke from troubled dreams, he found\nhimself transformed in his bed into a horrible vermin.  He lay on\nhis armour-like back, and if he lifted his head a little he could\nsee his brown belly, slightly domed and divided by arches into stiff\nsections.  The bedding was hardl'

## Tokenize

### Using `nltk`

In [40]:
from nltk.tokenize import word_tokenize

text_tokens = word_tokenize(text_lines)
print(len(text_tokens), text_tokens)

224 ['One', 'morning', ',', 'when', 'Gregor', 'Samsa', 'woke', 'from', 'troubled', 'dreams', ',', 'he', 'found', 'himself', 'transformed', 'in', 'his', 'bed', 'into', 'a', 'horrible', 'vermin', '.', 'He', 'lay', 'on', 'his', 'armour-like', 'back', ',', 'and', 'if', 'he', 'lifted', 'his', 'head', 'a', 'little', 'he', 'could', 'see', 'his', 'brown', 'belly', ',', 'slightly', 'domed', 'and', 'divided', 'by', 'arches', 'into', 'stiff', 'sections', '.', 'The', 'bedding', 'was', 'hardly', 'able', 'to', 'cover', 'it', 'and', 'seemed', 'ready', 'to', 'slide', 'off', 'any', 'moment', '.', 'His', 'many', 'legs', ',', 'pitifully', 'thin', 'compared', 'with', 'the', 'size', 'of', 'the', 'rest', 'of', 'him', ',', 'waved', 'about', 'helplessly', 'as', 'he', 'looked', '.', '``', 'What', "'s", 'happened', 'to', 'me', '?', "''", 'he', 'thought', '.', 'It', 'was', "n't", 'a', 'dream', '.', 'His', 'room', ',', 'a', 'proper', 'human', 'room', 'although', 'a', 'little', 'too', 'small', ',', 'lay', 'peacefu

### Using `spaCy`

In [41]:
# placeholder

### Using `gensim`

In [42]:
# placeholder

## Stopwords Removal

Tokens from the original text

In [43]:
print(len(text_tokens), text_tokens)

224 ['One', 'morning', ',', 'when', 'Gregor', 'Samsa', 'woke', 'from', 'troubled', 'dreams', ',', 'he', 'found', 'himself', 'transformed', 'in', 'his', 'bed', 'into', 'a', 'horrible', 'vermin', '.', 'He', 'lay', 'on', 'his', 'armour-like', 'back', ',', 'and', 'if', 'he', 'lifted', 'his', 'head', 'a', 'little', 'he', 'could', 'see', 'his', 'brown', 'belly', ',', 'slightly', 'domed', 'and', 'divided', 'by', 'arches', 'into', 'stiff', 'sections', '.', 'The', 'bedding', 'was', 'hardly', 'able', 'to', 'cover', 'it', 'and', 'seemed', 'ready', 'to', 'slide', 'off', 'any', 'moment', '.', 'His', 'many', 'legs', ',', 'pitifully', 'thin', 'compared', 'with', 'the', 'size', 'of', 'the', 'rest', 'of', 'him', ',', 'waved', 'about', 'helplessly', 'as', 'he', 'looked', '.', '``', 'What', "'s", 'happened', 'to', 'me', '?', "''", 'he', 'thought', '.', 'It', 'was', "n't", 'a', 'dream', '.', 'His', 'room', ',', 'a', 'proper', 'human', 'room', 'although', 'a', 'little', 'too', 'small', ',', 'lay', 'peacefu

### Using `nltk`

In [44]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
# print(len(stop_words),stop_words)

In [45]:
filtered_tokens = [word for word in text_tokens if word.lower() not in stop_words]
print(len(filtered_tokens), filtered_tokens)

132 ['One', 'morning', ',', 'Gregor', 'Samsa', 'woke', 'troubled', 'dreams', ',', 'found', 'transformed', 'bed', 'horrible', 'vermin', '.', 'lay', 'armour-like', 'back', ',', 'lifted', 'head', 'little', 'could', 'see', 'brown', 'belly', ',', 'slightly', 'domed', 'divided', 'arches', 'stiff', 'sections', '.', 'bedding', 'hardly', 'able', 'cover', 'seemed', 'ready', 'slide', 'moment', '.', 'many', 'legs', ',', 'pitifully', 'thin', 'compared', 'size', 'rest', ',', 'waved', 'helplessly', 'looked', '.', '``', "'s", 'happened', '?', "''", 'thought', '.', "n't", 'dream', '.', 'room', ',', 'proper', 'human', 'room', 'although', 'little', 'small', ',', 'lay', 'peacefully', 'four', 'familiar', 'walls', '.', 'collection', 'textile', 'samples', 'lay', 'spread', 'table', '-', 'Samsa', 'travelling', 'salesman', '-', 'hung', 'picture', 'recently', 'cut', 'illustrated', 'magazine', 'housed', 'nice', ',', 'gilded', 'frame', '.', 'showed', 'lady', 'fitted', 'fur', 'hat', 'fur', 'boa', 'sat', 'upright', 

### Using `spaCy`

In [46]:
# placeholder

### Using `gensim`

In [47]:
# placeholder

## Lemmatization

### Using `nltk`

In [48]:
# placeholder

### Using `spaCy`

In [49]:
filtered_text = " ".join([token for token in filtered_tokens])
lemmas = [word.lemma_ for word in nlp(filtered_text)]
print(len(lemmas), lemmas)

135 ['one', 'morning', ',', 'Gregor', 'Samsa', 'wake', 'troubled', 'dream', ',', 'find', 'transform', 'bed', 'horrible', 'vermin', '.', 'lie', 'armour', '-', 'like', 'back', ',', 'lift', 'head', 'little', 'could', 'see', 'brown', 'belly', ',', 'slightly', 'dome', 'divided', 'arch', 'stiff', 'section', '.', 'bed', 'hardly', 'able', 'cover', 'seem', 'ready', 'slide', 'moment', '.', 'many', 'leg', ',', 'pitifully', 'thin', 'compare', 'size', 'rest', ',', 'wave', 'helplessly', 'look', '.', '`', '`', 'be', 'happen', '?', "''", 'think', '.', 'not', 'dream', '.', 'room', ',', 'proper', 'human', 'room', 'although', 'little', 'small', ',', 'lie', 'peacefully', 'four', 'familiar', 'wall', '.', 'collection', 'textile', 'sample', 'lay', 'spread', 'table', '-', 'samsa', 'travel', 'salesman', '-', 'hang', 'picture', 'recently', 'cut', 'illustrate', 'magazine', 'house', 'nice', ',', 'gild', 'frame', '.', 'show', 'lady', 'fit', 'fur', 'hat', 'fur', 'boa', 'sit', 'upright', ',', 'raise', 'heavy', 'fur'

### Using `gensim`

In [50]:
# placeholder

## Stemming

### Using `nltk`

In [51]:
# placeholder

### Using `spaCy`

In [52]:
# placeholder

### Using `gensim`

In [53]:
# placeholder